# Extracting runs from one DB file to another

This notebook shows how to use the `extract_runs_into_db` function to extract runs from DB file (the source DB) into another DB file (the target DB). If the target DB does not exist, it will be created. The runs are **NOT** removed from the original DB file; they are copied over.


## Setup

Let us set up a DB file with some runs in it.

In [1]:
import os 

import numpy as np

from qcodes.dataset.sqlite_base import connect
from qcodes.dataset.database_extract_runs import extract_runs_into_db
from qcodes.dataset.experiment_container import Experiment, load_experiment_by_name
from qcodes.tests.instrument_mocks import DummyInstrument
from qcodes.dataset.measurements import Measurement
from qcodes import Station

In [2]:
source_path = os.path.join(os.getcwd(), 'extract_runs_notebook_source.db')
target_path = os.path.join(os.getcwd(), 'extract_runs_notebook_target.db')

In [3]:
source_conn = connect(source_path)
target_conn = connect(target_path)

Upgrading database: : 0it [00:00, ?it/s]
Upgrading database: : 0it [00:00, ?it/s]


In [4]:
exp = Experiment(name='extract_runs_experiment',
                 sample_name='no_sample',
                 conn=source_conn)

my_inst = DummyInstrument('my_inst', gates=['voltage', 'current'])
station = Station(my_inst)

In [5]:
meas = Measurement(exp=exp)
meas.register_parameter(my_inst.voltage)
meas.register_parameter(my_inst.current, setpoints=(my_inst.voltage,))

#Add 10 runs with gradually more and more data

for run_id in range(1, 11):
    with meas.run() as datasaver:
        for step, noise in enumerate(np.random.randn(run_id)):
            datasaver.add_result((my_inst.voltage, step),
                                 (my_inst.current, noise))

Starting experimental run with id: 1
Starting experimental run with id: 2
Starting experimental run with id: 3
Starting experimental run with id: 4
Starting experimental run with id: 5
Starting experimental run with id: 6
Starting experimental run with id: 7
Starting experimental run with id: 8
Starting experimental run with id: 9
Starting experimental run with id: 10


## Extraction

Now let us extract runs 3 and 7 into our desired target DB file. All runs must come from the same experiment. To extract runs from different experiments, call the function several times.

The function will look in the target DB to see if an experiment with matching attributes already exists. If not, such an experiment is created.

In [6]:
extract_runs_into_db(source_path, target_path, 3, 7)

In [7]:
target_exp = load_experiment_by_name(name='extract_runs_experiment', conn=target_conn)

In [8]:
target_exp.data_sets

<bound method Experiment.data_sets of extract_runs_experiment#no_sample#1@C:\Users\wihpniel/src/Qcodes/docs/examples/DataSet/experiments.db
-----------------------------------------------------------------------------------------------------
1-results-1-my_inst_voltage,my_inst_current-3
2-results-2-my_inst_voltage,my_inst_current-7>

The last number printed in each line is the number of data points. As expected, we get 3 and 7.

Note that the runs will have different `run_id`s in the new database. Their GUIDs are, however, the same (as they must be).

In [9]:
exp.data_set(3).guid

'aaaaaaaa-0c00-0007-0000-01675f5f9479'

In [10]:
target_exp.data_set(1).guid

'aaaaaaaa-0c00-0007-0000-01675f5f9479'